# Análise BigQuery - Mercado Libre (Samsung Galaxy S25)
___

### Notebook para responder as perguntas:
1) ¿Hay algún vendedor con múltiples publicaciones? En caso de que si, ¿con cuantas?
2) ¿Promedio de ventas por seller?
3) ¿Cuál es el precio promedio en dólares?
4) ¿Porcentaje de artículos con garantía?
5) ¿Métodos de Shipping que ofrecen?

### Instruções:
- Ajuste PROJECT_ID, DATASET e TABLE abaixo (ou configure via variáveis de ambiente).
- Garanta que a variável de ambiente GOOGLE_APPLICATION_CREDENTIALS esteja definida com a chave do service account.
- Executar células sequencialmente.

### Observações Adicionais:
Nos últimos meses, a API do Mercado Livre passou por alterações relevantes que impactaram diretamente o acesso a dados anteriormente públicos. Endpoints como /search e /items, antes amplamente acessíveis, passaram a exigir autenticação e, adicionalmente, foram limitados para retorno apenas de itens próprios, impossibilitando a consulta aberta a anúncios de terceiros. Essas mudanças inviabilizaram a extração completa das informações necessárias para o case exclusivamente via API oficial.

Diante desse cenário, foi necessário recorrer a alternativas para concluir o desafio. Considerando que o uso direto de scraping e crawling é bloqueado pelos mecanismos de segurança e privacidade do MeLi — demandando infraestrutura adicional com proxies e rotação de IPs — optou-se pelo uso de clientes de API especializados em scraping, viabilizando o acesso aos dados dentro do tempo disponível.

Portanto, houve diferença quanto aos retornos esperados, uma vez que alterou-se a API de origem, resultando na ausência de informações como métodos de envio disponíveis e produtos sob garantia, impactando as respostas das questões 4 e 5. Entretanto, a fim de demonstrar os conhecimentos necessários, elaborei cenários hipotéticos simulando a presença dessas colunas e apresentei como as trataria em código.

Essa abordagem permitiu a execução plena do case, respeitando os limites técnicos e prazos estabelecidos, ao mesmo tempo em que garantiu a demonstração das capacidades técnicas esperadas.



In [16]:
# Configuração inicial
import os
import math
import requests
import pandas as pd
from google.cloud import bigquery
from datetime import datetime

# Ajuste estes valores conforme seu projeto/dataset/tabela
PROJECT_ID = os.getenv("GCP_PROJECT_ID", "seu-projeto-gcp")
DATASET = os.getenv("BQ_DATASET", "mercado_libre")
TABLE = os.getenv("BQ_TABLE", "products_samsung_s25")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../meli-etl-key.json"

BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

print("Usando tabela:", BQ_TABLE)

# Inicializar cliente BigQuery
client = bigquery.Client(project=PROJECT_ID)

# Helper: executar query e retornar DataFrame
def run_query(sql: str, max_results: int = None) -> pd.DataFrame:
    job = client.query(sql)
    df = job.result().to_dataframe(create_bqstorage_client=True, progress_bar_type=None)
    if max_results is not None:
        return df.head(max_results)
    return df

Usando tabela: `meli-etl-476805.mercado_libre.products_samsung_s25`


### 1) ¿Hay algún vendedor con múltiples publicaciones? En caso de que si, ¿con cuantas?
 - Definimos "publicación" como uma linha/tabela entry.
 - Buscar sellers com COUNT(*) > 1, listar quantidade de publicações por seller,
   e mostrar top sellers.


In [ ]:
sql_sellers_multi = f"""
SELECT
  seller,
  COUNT(1) AS listings_count
FROM {BQ_TABLE}
WHERE seller IS NOT NULL AND seller != '' AND condition = 'New'
GROUP BY seller
HAVING COUNT(1) > 1
ORDER BY listings_count DESC
LIMIT 1000
"""
df_sellers_multi = run_query(sql_sellers_multi)
print("Sellers com múltiplas publicações (top 10):")
display(df_sellers_multi.head(10))

# Resumo: quantos sellers têm múltiplas publicações e totais
sql_sellers_multi_summary = f"""
WITH counts AS (
  SELECT seller, COUNT(1) AS listings_count
  FROM {BQ_TABLE}
  WHERE seller IS NOT NULL AND seller != '' AND condition = 'New'
  GROUP BY seller
)
SELECT
  COUNT(1) AS sellers_with_multiple_listings,
  SUM(listings_count) AS total_listings_of_these_sellers,
  AVG(listings_count) AS avg_listings_per_seller_with_multiple
FROM counts
WHERE listings_count > 1
"""
df_sellers_multi_summary = run_query(sql_sellers_multi_summary)
display(df_sellers_multi_summary)

### 2) ¿Promedio de ventas por seller?
- Existem duas interpretações:

  A) Promedio de "sellCount" por seller (média das vendas reportadas por cada publicação).

  B) Promedio total de ventas por seller (soma de sellCount por seller, depois média entre sellers).

In [ ]:
# A) Media de sellCount por listing (considerando apenas linhas com sellCount)
sql_avg_sellcount_per_listing = f"""
SELECT
  AVG(sellCount) AS avg_sellcount_per_listing
FROM {BQ_TABLE}
WHERE sellCount IS NOT NULL AND condition = 'New'
"""
df_avg_a = run_query(sql_avg_sellcount_per_listing)
display(df_avg_a)

# B) Para cada seller, soma sellCount; depois média entre sellers (usando APPROX_QUANTILES pra mediana)
sql_avg_total_sales_per_seller = f"""
WITH seller_totals AS (
  SELECT
    seller,
    SUM(COALESCE(sellCount, 0)) AS total_sold,
    COUNT(1) AS listings
  FROM {BQ_TABLE}
  WHERE seller IS NOT NULL AND seller != '' AND condition = 'New'
  GROUP BY seller
)
SELECT
  COUNT(1) AS num_sellers,
  AVG(total_sold) AS avg_total_sales_per_seller,
  APPROX_QUANTILES(total_sold, 100)[OFFSET(50)] AS median_total_sales_per_seller
FROM seller_totals;
"""
df_avg_b = run_query(sql_avg_total_sales_per_seller)
display(df_avg_b)

# Se quiser ver top sellers por total_sold:
sql_top_sellers = f"""
SELECT seller, SUM(COALESCE(sellCount,0)) AS total_sold, COUNT(1) AS listings
FROM {BQ_TABLE}
WHERE seller IS NOT NULL AND seller != '' AND condition = 'New'
GROUP BY seller
ORDER BY total_sold DESC
LIMIT 50
"""
df_top_sellers = run_query(sql_top_sellers)
print("Top sellers por vendas totais (top 10):")
display(df_top_sellers.head(10))

### 3) ¿Cuál es el precio promedio en dólares?
- A tabela contém 'price' (FLOAT) e 'currency' (STRING).
- Estratégia:
  * Se currency = 'USD' -> usa price direto.
  * Se currency = 'ARS' -> converte usando taxa de câmbio ARS->USD.
  * Outros currencies -> descartamos.

In [15]:
ars_to_usd = 0.00069

# Query que converte preços para USD (aplica apenas a currency 'ARS' e 'USD')
sql_price_usd = f"""
SELECT
  COUNT(1) AS total_items_considered,
  AVG(CASE
        WHEN currency = 'USD' THEN price
        WHEN currency = 'ARS' THEN SAFE_DIVIDE(price, {1.0/ars_to_usd if ars_to_usd != 0 else 1})
        -- Note: SAFE_DIVIDE used in case of issues; here we compute price in USD by dividing by (ARS per USD).
        ELSE NULL
      END) AS avg_price_usd,
  PERCENTILE_CONT(CASE
        WHEN currency = 'USD' THEN price
        WHEN currency = 'ARS' THEN SAFE_DIVIDE(price, {1.0/ars_to_usd if ars_to_usd != 0 else 1})
        ELSE NULL
      END, 0.5) OVER() AS median_price_usd
FROM {BQ_TABLE}
WHERE price IS NOT NULL AND currency IS NOT NULL  AND condition = 'New'
"""
# Observação: a formula acima usa 1/(ARS->USD) para converter; se ARS->USD = 0.005 (1 ARS = 0.005 USD),
# então price_in_usd = price * 0.005. Para manter legível, definimos denom = 1/(rate) quando queríamos dividir.
# Para evitar confusão, vamos calcular no pandas em seguida com taxa correta.

df_price_sample = run_query(f"""
SELECT price, currency FROM {BQ_TABLE} WHERE price IS NOT NULL  AND condition = 'New' LIMIT 1000
""", max_results=1000)
# Converter em pandas para aplicar taxa de forma explícita
def convert_price_to_usd(row, rate):
    try:
        if row["currency"] == "USD":
            return float(row["price"])
        if row["currency"] == "ARS":
            # rate = 1 ARS = rate USD
            return float(row["price"]) * rate
    except Exception:
        return None

df_price_sample["price_usd"] = df_price_sample.apply(lambda r: convert_price_to_usd(r, ars_to_usd), axis=1)
# Calcular média com pandas usando amostra (para demonstrar)
print("Exemplo (amostra) - média convertida (USD):", df_price_sample["price_usd"].dropna().mean())

# Para cálculo robusto em BigQuery (aplicando rate): vamos usar rate como multiplicador (price * rate when ARS)
rate = ars_to_usd
sql_price_usd_bq = f"""
SELECT
  COUNT(1) AS total_items_considered,
  AVG(CASE
        WHEN currency = 'USD' THEN price
        WHEN currency = 'ARS' THEN price * {rate}
        ELSE NULL
      END) AS avg_price_usd,
  APPROX_QUANTILES(CASE
        WHEN currency = 'USD' THEN price
        WHEN currency = 'ARS' THEN price * {rate}
        ELSE NULL
      END, 100)[OFFSET(50)] AS median_price_usd
FROM {BQ_TABLE}
WHERE price IS NOT NULL AND currency IN ('USD','ARS') AND condition = 'New'
"""
df_price_usd_bq = run_query(sql_price_usd_bq)
display(df_price_usd_bq)

Exemplo (amostra) - média convertida (USD): 1352.2851992680849


,total_items_considered,avg_price_usd,median_price_usd
0,47,1352.285199,1276.5


### 4) ¿Porcentaje de artículos con garantía?
- Considerando o campo `warranty` (STRING), que o item tem garantia se `warranty` não é nulo e não é vazio.
- Calculo percentagem sobre total de registros com informação de warranty (ou sobre total de rows).

In [17]:
sql_warranty_pct = f"""
SELECT
  COUNT(1) AS total_items,
  SUM(CASE WHEN warranty IS NOT NULL AND TRIM(warranty) <> '' THEN 1 ELSE 0 END) AS items_with_warranty,
  SAFE_DIVIDE(SUM(CASE WHEN warranty IS NOT NULL AND TRIM(warranty) <> '' THEN 1 ELSE 0 END), COUNT(1)) AS pct_with_warranty
FROM {BQ_TABLE}
"""
#df_warranty = run_query(sql_warranty_pct)
# formatar percent
#df_warranty["pct_with_warranty_percent"] = df_warranty["pct_with_warranty"] * 100
#display(df_warranty)

### 5) ¿Métodos de Shipping que ofrecen?
- Considerando verificar possíveis campos que contenham a palavra 'ship' ou 'envio'.
- Se não houver coluna explícita, procurar na coluna `description` por termos comuns (MercadoEnvíos, envío, retiro, envío gratis, pickup, "full").
- Em seguida agrupar e contar.

In [ ]:
# 5a) listar colunas para conferir se existe campo relacionado a shipping
table = client.get_table(f"{PROJECT_ID}.{DATASET}.{TABLE}")
print("Colunas da tabela:", [f.name for f in table.schema])

# 5b) Se houver coluna 'shipping' ou similar, mostrar distinct
shipping_like_cols = [c.name for c in table.schema if any(k in c.name.lower() for k in ("ship", "envio", "shipping", "shipment"))]
print("Colunas com nome relacionado a shipping:", shipping_like_cols)

if shipping_like_cols:
    for col in shipping_like_cols:
        sql_shipping_col = f"""
        SELECT {col} AS shipping_value, COUNT(1) AS cnt
        FROM {BQ_TABLE}
        GROUP BY {col}
        ORDER BY cnt DESC
        LIMIT 100
        """
        df_shipping_col = run_query(sql_shipping_col)
        print(f"Distinct values for {col}:")
        display(df_shipping_col)
else:
    print("Nenhuma coluna explícita relacionada a shipping encontrada. Buscando por keywords na descrição...")

# Buscar keywords na descrição
keywords = ["envio", "envío", "mercadoenvíos", "mercadoenvios", "envío gratis", "retiro", "retiro en", "envío gratis", "mercadoenvios full", "full", "shipping", "pickup", "envio gratis"]
# Construir expressão REGEXP_CONTAINS ORs
regex_expr = "|".join([kw.replace("'", "\\'") for kw in keywords])
sql_search_shipping = f"""
SELECT
  CASE
    WHEN REGEXP_CONTAINS(LOWER(COALESCE(description,'')), r'({regex_expr})') THEN 'mentions_shipping_keywords'
    ELSE 'no_keywords'
  END AS shipping_kw,
  COUNT(1) as cnt
FROM {BQ_TABLE}
GROUP BY shipping_kw
"""
df_shipping_kw = run_query(sql_search_shipping)
display(df_shipping_kw)

# Para extrair snippets e estimar métodos mais usados, buscar frases contendo keywords
sql_snippets = f"""
SELECT
  description,
  url,
  seller,
  REGEXP_EXTRACT(LOWER(COALESCE(description, '')), r'(env[ií]o[s]?\\s+gratis|retiro\\s+en\\s+[a-z0-9\\s]+|mercadoenv[ií]os|full|shipping|pickup)') AS snippet
FROM {BQ_TABLE}
WHERE REGEXP_CONTAINS(LOWER(COALESCE(description, '')), r'({regex_expr})')
LIMIT 200
"""
#df_snippets = run_query(sql_snippets)
#print("Exemplos de snippets onde aparecem referências a shipping (até 200 amostras):")
#display(df_snippets.head(50))

## Agrupar por snippet para ver métodos frequentes
#df_snippets["snippet_clean"] = df_snippets["snippet"].fillna("").str.strip()
#shipping_summary = df_snippets["snippet_clean"].value_counts().reset_index()
#shipping_summary.columns = ["snippet", "count"]
#display(shipping_summary.head(50))